# 패스트텍스트(FastText)

단어를 벡터로 만드는 또 다른 방법으로 페이스북에서 개발한 FastText. Word2Vec의 확장. FastText는 하나의 단어 안에도 여러 단어들이 존재하는 것으로 간주(subword)<br>
- n = 3인 경우:
<ap, app, ppl, ple, le>

In [1]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [2]:
pip list | grep gensim

gensim                             4.3.3


In [4]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [5]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7bb08b257850>)

In [6]:
targetXML = open("ted_en-20160408.xml", "r", encoding="UTF8")
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거
# 해당 코드는 괄호로 구성된 내용을 제거
content_text = re.sub(r"[)]*", "", parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행
result = [word_tokenize(sentence) for sentence in normalized_text]

In [7]:
print("총 샘플의 개수: {}".format(len(result)))

총 샘플의 개수: 273648


In [8]:
from gensim.models import Word2Vec, FastText

In [11]:
model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

In [12]:
model.wv.most_similar("electrofishing")

KeyError: "Key 'electrofishing' not present in vocabulary"

In [13]:
model = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [15]:
model.wv.most_similar("electrofishing")

# subword 를 이용하기에 OOV도 유사도 비교가 가능함!

[('electrolux', 0.8603139519691467),
 ('electrolyte', 0.854586124420166),
 ('electroshock', 0.8506245017051697),
 ('electro', 0.8459041118621826),
 ('electroencephalogram', 0.823811948299408),
 ('electrogram', 0.8233461976051331),
 ('electronic', 0.8160528540611267),
 ('electric', 0.8153147101402283),
 ('electrochemical', 0.808189332485199),
 ('electron', 0.8034310340881348)]